# NeuroNexus Allego file reading tutorial

## Imports

make sure this notebook is in the same directory as the `allego_file_reader.py` file


In [ ]:
import os
from pathlib import Path
import allego_file_reader as afr

In [ ]:
target_dir = "data" # data directory
p = Path(target_dir).expanduser()

In [ ]:
all_xdat_datasource_names = [Path(elem.stem).stem for elem in list(p.glob('**/*xdat.json'))]
print(all_xdat_datasource_names)
base_name = all_xdat_datasource_names[0] # first base name
base_name = 'allego_2__uid1120-14-16-26'
print("Will work with file %s" % base_name)
file_path = str(Path(target_dir, base_name))

## Metadata (channel mapping)

In [ ]:
meta = afr.read_allego_xdat_metadata(file_path) # read metadata

## Signals

In [ ]:
time_range = afr.get_allego_xdat_time_range(file_path)
time_range

In [ ]:
time_start = None
time_end = None

In [ ]:
pri_signals, timestamps, time_samples = afr.read_allego_xdat_pri_signals(file_path, 
                                                                         time_start, time_end)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.subplot(2,2,(1,2))
id = 30
plt.plot(pri_signals[id]) # channels go 0..31
plt.ylim([-0,100])
plt.subplot(223)

plt.plot(pri_signals[id]) # channels go 0..31
plt.xlim([1e6,1e6 + 1000])
plt.ylim([-100,100])

plt.subplot(224)
plt.plot(pri_signals[id]) # channels go 0..31

plt.xlim([4*1e6,4e6 + 1000])
plt.ylim([-100,100])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq

id = 21  # channel number
sig = pri_signals[id]

# sampling frequency
fs = 30000.0   
dt = 1.0 / fs

before_start = int(1e6)
before_end   = int(2e6)

after_start  = int(3.5e6)
after_end    = int(4.5e6)

before_seg = sig[before_start:before_end]
after_seg  = sig[after_start:after_end]

plt.figure(figsize=(14,10))

# full signal
plt.subplot(221)
plt.plot(sig, linewidth=0.7)
plt.title(f"Channel {id} — Full Signal")
plt.ylim([-10,100])
plt.xlabel("Sample")
plt.ylabel("Amplitude")

# before 
plt.subplot(223)
t_before = np.arange(len(before_seg)) * dt   # seconds
#plt.plot(t_before, before_seg, linewidth=0.7)
plt.plot(before_seg)
plt.xlabel("Sample index")
plt.title("Before EDTA")
plt.ylim([-100,100])
#plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")

# after
plt.subplot(224)
t_after = np.arange(len(after_seg)) * dt
#plt.plot(t_after, after_seg, linewidth=0.7, color='orange')
plt.plot(after_seg)
plt.xlabel("Sample index")
plt.title("After EDTA")
plt.ylim([-100,100])
#plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")

# FFT
plt.subplot(222)
N = len(before_seg)

freqs = rfftfreq(N, d=dt)   # now in Hz

plt.plot(freqs, np.abs(rfft(before_seg)), label='Before')
plt.plot(freqs, np.abs(rfft(after_seg)), label='After', alpha=0.7)
plt.title("FFT Comparison")
plt.xlabel("Frequency (Hz)")      # updated label
plt.ylabel("Magnitude")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import rfft, rfftfreq

num_channels = 31
window = 50000  # size of window to compare
fs = 30000  # sampling rate unknown → relative freq still OK

before_start = int(1e6)
after_start  = int(3.5e6)

def amplitude_metrics(x):
    return {
        "mean": np.mean(x),
        "std": np.std(x),
        "p2p": np.ptp(x),
        "rms": np.sqrt(np.mean(x**2))
    }

def frequency_metrics(x, fs=1):
    x = x - np.mean(x)
    N = len(x)
    X = rfft(x)
    freqs = rfftfreq(N, d=1/fs)
    mag = np.abs(X)

    peak_freq = freqs[np.argmax(mag)]
    centroid  = np.sum(freqs * mag) / np.sum(mag)
    power     = np.sum(mag**2)

    return {
        "peak_freq": peak_freq,
        "centroid": centroid,
        "power": power
    }


rows = []

for ch in range(num_channels):
    sig = pri_signals[ch]

    before = sig[before_start:before_start + window]
    after  = sig[after_start:after_start + window]

    amp_b = amplitude_metrics(before)
    amp_a = amplitude_metrics(after)

    freq_b = frequency_metrics(before, fs)
    freq_a = frequency_metrics(after, fs)

    row = {
        "channel": ch,

        # amplitude
        "mean_before": amp_b["mean"],
        "mean_after": amp_a["mean"],
        "std_before": amp_b["std"],
        "std_after": amp_a["std"],
        "p2p_before": amp_b["p2p"],
        "p2p_after": amp_a["p2p"],
        "rms_before": amp_b["rms"],
        "rms_after": amp_a["rms"],

        # frequency
        "peak_freq_before": freq_b["peak_freq"],
        "peak_freq_after": freq_a["peak_freq"],
        "centroid_before": freq_b["centroid"],
        "centroid_after": freq_a["centroid"],
        "power_before": freq_b["power"],
        "power_after": freq_a["power"],
    }

    rows.append(row)

df = pd.DataFrame(rows)
df
#df.to_csv("channel_analysis.csv", index=False)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.plot(df["channel"], df["mean_after"] - df["mean_before"], marker='o')
plt.axhline(0, color='k', linestyle='--')
plt.title("Change in mean (After - Before)")
plt.xlabel("Channel")
plt.ylabel("Δ mean")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df["channel"], df["std_after"] - df["std_before"], marker='o')
plt.axhline(0, color='k', linestyle='--')
plt.title("Change in Standard Deviation (After - Before)")
plt.xlabel("Channel")
plt.ylabel("Δ Standard Deviation")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df["channel"], df["rms_after"] - df["rms_before"], marker='o')
plt.axhline(0, color='k', linestyle='--')
plt.title("Change in Root Mean Square (After - Before)")
plt.xlabel("Channel")
plt.ylabel("Δ rms")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df["channel"], df["p2p_after"] - df["p2p_before"], marker='o')
plt.axhline(0, color='k', linestyle='--')
plt.title("Change in Peak to Peak (After - Before)")
plt.xlabel("Channel")
plt.ylabel("Δ p2p")
plt.show()